In [36]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [37]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [38]:
# let's look at the first 1000 chaacters 

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [39]:
 # unique characters in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print( ''.join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [40]:
# def encode(s):
#     return [stoi[c] for c in s]

# def decode(l):
#     return ''.join([itos[i] for i in l])

In [41]:
# create mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encode: take a string, output a list of integers
decode = lambda l: ''.join ([itos[i] for i in l]) # decoder: take a list of integers,output a string

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [42]:
# lets now encode the entire text dataset and store it into a torch.Tensor
import torch
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])
# encoded_text

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [43]:
# let's now split up the data into train and validation  sets

n = int(0.9 * len(data))
train_data = data[:n] # first 90% will be train 
val_data = data[n:]

In [44]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
y

tensor([18, 47, 56, 57, 58,  1, 15, 47])


tensor([47, 56, 57, 58,  1, 15, 47, 58])

In [45]:
for t in range(block_size):
 
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [46]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel ?
block_size = 8 # what is the maximum context length for prediction ?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("--------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}" )

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44

In [47]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) aaray of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step

            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim =1) #(B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [48]:
# create a pyTorch optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [49]:
batch_size = 32

for steps in range(30000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.467581272125244


In [50]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long),max_new_tokens=500)[0].tolist()))




Heofl go? be's my bermen we, E: a heawn?


Wes int cowh abldave ancowad l sh.


UClly.
Wh
'de nd, aiut;
Winghic
Lot yod aucacat t y hen d s w?
thouryor, che nin me whaineay f I ath s t'ly minos; bye
LINothis eldin;


Cotitree y m we KEEESes hi'laia at thiforleed, Brveyaced dot.
CEORCLAnng.
TI f'd; fedin thatho in, LAnd,
CONGoous, hat fois ind, fouexe, t d bethaketeerus wisous. l hirler'sthalathe I's t mbe telame cashofr aimolilake, sindin faivendshe fikis h bonst shodladre IORGSTalestheeseag a


### The mathematical trick in self -attention

In [51]:

torch.manual_seed(1337)

B,T,C = 4,8,2
x = torch.randn(B,T,C)
print(x.shape)

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, dim=0)





torch.Size([4, 8, 2])


In [52]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a =")
print(a)
print('----')
print('b =')
print(b)
print('----')
print('c =')
print(c)

a =
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
----
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c =
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [53]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [54]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a =")
print(a)
print('----')
print('b =')
print(b)
print('----')
print('c =')
print(c)

a =
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c =
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [55]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) ------> (B, T, T) @ (B, T, C) ------> (B, T, C) |(T, T) is broadcasted to (B, T, T)|
print(torch.allclose(xbow, xbow2))

True


In [56]:
# version 3: use softmax

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [57]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [58]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [59]:
xbow3 = wei @ x
torch.allclose(xbow2,xbow3)

True

#### single head of self-attention

In [70]:
## version 4: self-attention
torch.manual_seed(1337)
# This is used as decoder as the token is allowed to pass info only with the previous token only
# This is self-attention because the key, query and value all come from same source which is x here
# Cross attention is when the key is from x and query and value is coming from some other external source
B,T,C = 4, 8, 32 # batch, time, chanels
x = torch.randn(B, T, C)

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias =False)
value = nn.Linear(C, head_size, bias =False)
k = key(x)
q = query(x) 
# Matrix multiplication requires that the inner dimensions of the two matrices being multiplied match. Specifically, 
# for two matrices A and B with shapes (a, b, c) and (a, c, d), respectively, you can multiply them if their inner dimensions match (c in this case).
wei = q @ k.transpose(-2, -1) # we are performing batch wise multiplication so that we get data independent 
# If we do the direct multiplication we won't get the dot product i.e we are transposing
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [64]:
wei

tensor([[[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
         [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
         [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],

        [[0.0443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0042, 0.0375, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0560, 0.0210, 0.2496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3679, 0.1441, 0.4929, 0.0438, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0088, 0.1052, 0.0604, 0.5847, 0.2046, 0.0000, 0.0000, 0.0000],
         [0.0367, 0.089

 both k.transpose(-2, -1) and k.transpose(-2, 1) can result in the same shape when applied to a tensor of shape (B, T, C) (e.g., they both result in (B, C, T) if k originally has shape (B, T, C)). However, the reason for specifically using k.transpose(-2, -1) lies in what is being transposed and why.

The Context of Self-Attention

In self-attention, we calculate the attention weights using the dot product of the query (Q) and key (K) matrices:

 \text{attention\_weights} = Q \times K^T 

	•	Q has shape (B, T, head_size).
	•	K has shape (B, T, head_size).

To perform the dot product between Q and K, we need to transpose the last two dimensions of K to get a shape of (B, head_size, T).

Why Use .transpose(-2, -1)?

	•	.transpose(-2, -1) is used because it explicitly swaps the last two dimensions, converting K from (B, T, head_size) to (B, head_size, T).
	•	This ensures that when you perform the matrix multiplication q @ k.transpose(-2, -1), the dot product happens between the sequence length T across the queries and keys, producing a matrix of shape (B, T, T).

Why Not Use .transpose(-2, 1)?

Although .transpose(-2, 1) can also produce a tensor of shape (B, head_size, T), it’s not used in this case because:

	•	It is more general and clearer to use .transpose(-2, -1) when dealing with attention operations, where we always want to transpose the last two dimensions.
	•	In other scenarios, .transpose(-2, 1) could lead to unintended behavior when applied to tensors with more or fewer than three dimensions.

Summary

The choice of .transpose(-2, -1) is intentional because it specifically handles transposing the last two dimensions, which is what’s needed for self-attention. While both operations yield the same shape here, the correct conceptual operation in self-attention is to swap the last two dimensions.

In [68]:
## TO use the above self attention as encoder block where each token is allowed to share inforamation with every other
## version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4, 8, 32 # batch, time, chanels
x = torch.randn(B, T, C)

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias =False)
value = nn.Linear(C, head_size, bias =False)
k = key(x)
q = query(x)
# Matrix multiplication requires that the inner dimensions of the two matrices being multiplied match. Specifically, 
# for two matrices A and B with shapes (a, b, c) and (a, c, d), respectively, you can multiply them if their inner dimensions match (c in this case).
wei = q @ k.transpose(-2, -1) # we are performing batch wise multiplication so that we get data independent 
# If we do the direct multiplication we won't get the dot product i.e we are transposing
tril = torch.tril(torch.ones(T,T))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

#### Notes:
##### 1.Attention is a comminication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sumfrom all nodes that point to them, with the data-dependent weights.
##### 2.There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
##### 3.Each example across batch dimmension is of course processed completely independently and never "talk" to each other.
##### 4.In an "encoder" attention block just delete the single line that does masking eith tril, allowing all tokens to communicate. This block above is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modelling.
##### 5."self-attention" just means that the keys and values are produced from the same source as queries.In "cross-attention", the queries still get produced from x, but the keys and values comes from some other external source(e.g. an encoder model)
##### 6."Scaled" attention additional divides wei by1/sqrt(head_size). This makes it so when Q.K are unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below
##### 7. Because we aare passing wei to softmax if the num is too +ve or too -ve it will converge towards one hot vectors and when we multiply with bigger number it will sharpen towards the max and we will be agrregating information froma a single node. i.e we divide to diffuse the wei before multiplying with v

In [72]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [73]:
k.var()

tensor(0.9006)

In [74]:
q.var()

tensor(1.0037)

In [75]:
wei.var()

tensor(0.9957)

In [79]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [80]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [3]:
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    
    def __call__(self, x):
        # calculate the forward pairs
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True, unbiased=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma + xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
import torch    
torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])